![rmotr](https://user-images.githubusercontent.com/7065401/52071918-bda15380-2562-11e9-828c-7f95297e4a82.png)
<hr style="margin-bottom: 40px;">

<img src="resources/bybit_logo.png"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>

# Bybit **position** portfolio place trades

Application for entering pending orders from csv files on Bybit futures cryptocurrency exchanges.

[Follow this data in a CSV Spreadsheet](data/position_trades.csv)

![purple-divider](resources/purple_divider.png)

## Hands on! 

In [1]:
import pandas as pd
from pybit.unified_trading import HTTP
from utils import load_config, transform_trades, place_trades_on_exchange, write_trades_to_audit_log

config = load_config("config.yaml")
ACCOUNT_TYPE = "bybitPositionAccount"

exchange = HTTP(
    testnet = config[ACCOUNT_TYPE]["bybitApi"]["testnet"],
    api_key = config[ACCOUNT_TYPE]["bybitApi"]["apiKey"],
    api_secret = config[ACCOUNT_TYPE]["bybitApi"]["secretKey"]
)

![green-divider](resources/green_divider.png)

## Loading data

In [2]:
trades_raw = pd.read_csv("data/position_trades.csv")
instruments_info = exchange.get_instruments_info(category="linear")

![green-divider](resources/green_divider.png)


## The data at a glance

In [3]:
trades_raw

,ticker,direction,entry_price,stop_loss
0,BTCUSDT,LONG,20000,19800


In [11]:
str(instruments_info)[:500]

"{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'linear', 'list': [{'symbol': '10000NFTUSDT', 'contractType': 'LinearPerpetual', 'status': 'Trading', 'baseCoin': '10000NFT', 'quoteCoin': 'USDT', 'launchTime': '1643007175000', 'deliveryTime': '0', 'deliveryFeeRate': '', 'priceScale': '6', 'leverageFilter': {'minLeverage': '1', 'maxLeverage': '12.50', 'leverageStep': '0.01'}, 'priceFilter': {'minPrice': '0.000005', 'maxPrice': '9.999990', 'tickSize': '0.000005'}, 'lotSizeFilter': {'maxOrderQ"

![green-divider](resources/green_divider.png)


## Transform data

In [13]:
trades = transform_trades(trades_raw, instruments_info, config, ACCOUNT_TYPE)

trades

,ticker,direction,entry_price,stop_loss,move,profit_target,position,position_usd,leverage,trigger_price,price_scale,qty_step,expected_profit,expected_loss
0,BTCUSDT,LONG,20000,19800,200,20200,0.003,50.0,11.11,20004.0,2,0.001,0.6,-0.6


![green-divider](resources/green_divider.png)


## Process data

In [15]:
write_trades_to_audit_log(trades, config[ACCOUNT_TYPE]["auditLogPath"])

place_trades_on_exchange(trades, exchange)



,trade_id,status
0,BTCUSDT_LONG,OK
